In [ ]:
import os, numpy as np, pandas as pd
import bokeh
import holoviews as hv
from bokeh.models import HoverTool, CustomJSHover
import xarray as xr
from oggm.utils import tolist
from collections import OrderedDict
import panel as pn
hv.extension('bokeh')

In [ ]:
from international import trads
language = 'en'

## Read the data 

Data sources:
- volume change: [Zekollari, H., Huss, M., and Farinotti, D.: Modelling the future evolution of glaciers in the European Alps under the EURO-CORDEX RCM ensemble, The Cryosphere, 13, 1125–1146, https://doi.org/10.5194/tc-13-1125-2019, 2019. ](https://www.the-cryosphere.net/13/1125/2019/)
- glacier locations: [Randolph Glacier Inventory](https://www.glims.org/RGI/)

In [ ]:
data = pd.read_hdf('./data/rgi62_era5_itmix_country_df.h5', 'df')
data['vol_itmix_km3'] = data['vol_itmix_m3'] * 1e-9

Select the Alps only:

In [ ]:
data = data.loc[data.O1Region == '11']
data = data.loc[(data.CenLon > 2.5) & (data.CenLat > 43.2)]
data.plot(kind='scatter', x='CenLon', y='CenLat');

In [ ]:
data_ts = []
data_ts_uq = []
for rcp in ['rcp26', 'rcp45', 'rcp85']:
    df = pd.read_csv('data/ZekollariHussFarinotti_TC2019_volume_{}.csv'.format(rcp), header=1, index_col=0).T
    df.columns = ['RGI60-11.{:05d}'.format(i) for i in df.columns]
    df = df[data.index]
    df.index = np.array(df.index).astype(int)
    data_ts.append(xr.DataArray(df, dims=['year', 'rgi_id']))
    df = pd.read_csv('data/ZekollariHussFarinotti_TC2019_volume_RCMspread_{}.csv'.format(rcp), header=1, index_col=0).T
    df.columns = ['RGI60-11.{:05d}'.format(i) for i in df.columns]
    df = df[data.index]
    df.index = np.array(df.index).astype(int)
    data_ts_uq.append(xr.DataArray(df, dims=['year', 'rgi_id']))


In [ ]:
ds = xr.concat(data_ts, dim='rcp')
ds.coords['rcp'] = ['RCP2.6', 'RCP4.5', 'RCP8.5']
ds_uq = xr.concat(data_ts_uq, dim='rcp')
ds_uq.coords['rcp'] = ['RCP2.6', 'RCP4.5', 'RCP8.5']

## Create the plot 

A multiline plot per selection of glaciers:

In [ ]:
hover = HoverTool(tooltips=[('Year', '@{year}'), 
                            ('Volume (% 2017)', '@{Volume (% 2017)}{int}'),
                            ('+σ', '@{+σ}{int}'),
                            ('-σ', '@{-σ}{int}'),
                           ], 
                  mode='mouse')

def sel_glaciers(rgi_ids):
    sel = ds.sel(rgi_id=tolist(rgi_ids)).sum(dim='rgi_id')
    sel_t = (ds + ds_uq).sel(rgi_id=tolist(rgi_ids)).sum(dim='rgi_id')
    sel_b = (ds - ds_uq).sel(rgi_id=tolist(rgi_ids)).sum(dim='rgi_id')
    sel_t = sel_t / sel.isel(year=0) * 100
    sel_b = sel_b / sel.isel(year=0) * 100
    sel = sel / sel.isel(year=0) * 100
    return sel, sel_b, sel_t

def make_curve(rgi_ids, rcp=None, line_dash=(0, 0), add_label='Selected region ', color='#30a2da'):
    
    sel, sel_b, sel_t = sel_glaciers(rgi_ids)
    df = sel.sel(rcp=rcp).to_dataframe(name='Volume (% 2017)')
    df['+σ'] = sel_t.sel(rcp=rcp).to_series().clip(0)
    df['-σ'] = sel_b.sel(rcp=rcp).to_series().clip(0)
    
    return hv.Curve(df, vdims=['Volume (% 2017)', 'rcp', '+σ', '-σ'], 
                    kdims=['year'], 
                    label=add_label+rcp).opts(tools=['hover'], line_dash=line_dash, color=color) 
              

def sel_overlay(rgi_ids, line_dash=(0, 0), add_label='Selected region: '):
    return hv.Overlay(
              make_curve(rgi_ids, rcp='RCP2.6', line_dash=line_dash, add_label=add_label, color='#30a2da') *
              make_curve(rgi_ids, rcp='RCP4.5', line_dash=line_dash, add_label=add_label, color='#e5ae38') *
              make_curve(rgi_ids, rcp='RCP8.5', line_dash=line_dash, add_label=add_label, color='#fc4f30')
            )

Define the regions:

In [ ]:
curve_dict = OrderedDict()
curve_dict['Entire Alps'] = sel_overlay(data.index)
for c in data.Country_Name.unique():
    if c == 'Slovenia':
        continue
    curve_dict[c] = sel_overlay(data.loc[data.Country_Name==c].index)
curve_dict['Oetztal/Pitzal'] = sel_overlay(['RGI60-11.00670', 'RGI60-11.00666', 'RGI60-11.00663', 'RGI60-11.00648', 'RGI60-11.00674'])

# This could be added at whish
# curve_dict['Brunnenkogelferner'] = sel_overlay('RGI60-11.00670')
# curve_dict['Mittelberferner'] = sel_overlay('RGI60-11.00666')
# curve_dict['Karlesferner'] = sel_overlay('RGI60-11.00663')
# curve_dict['Rettenbachferner'] = sel_overlay('RGI60-11.00648')
# curve_dict['Tiefenbachferner'] = sel_overlay('RGI60-11.00674')

In [ ]:
hmap = hv.HoloMap(curve_dict, kdims='Region', sort=False)
fplot = sel_overlay(data.index, line_dash=(4, 4), add_label='Entire Alps: ').opts(tools=['hover']) * hmap.opts(tools=['hover']) 
fplot = fplot.opts(width=900, height=500, fontsize={'title': 16, 'labels': 14, 'xticks': 12, 'yticks': 12, 'legend':12})

## Homepage texts and logos

In [ ]:
title = pn.pane.Markdown(sizing_mode='stretch_height', width=1080)
title.object = '<div style="font-size:38px; color: #326a86; font-weight: bold" >{}</div>'.format(trads['Title'][language])
#where to find this logo?
oggm_logo   = '<a href="http://edu.oggm.org"><img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/oggm_loupe.png" width=220></a>'
pn_logo = '<a href="https://panel.pyviz.org"><img src="https://panel.pyviz.org/_static/logo_stacked.png" width=46 height=39></a>'
holo_logo = '<a href="https://holoviz.org/"><img src="https://raw.githubusercontent.com/pyviz/holoviews/master/doc/_static/logo.png" width=46 height=39></a>'

In [ ]:
intro_text = trads['instruction'][language]
instruction = pn.pane.Markdown('<span style="color:#326a86;font-size:15px">'+intro_text+'</span>')
#sel_region = '<div>{}</div>'.format(trads['sel_region'][language])

In [ ]:
logos = pn.Row(pn_logo, holo_logo)
header = pn.Column(pn.Pane(oggm_logo), pn.Spacer(height=17), logos)
source = pn.pane.Markdown('Data source: [Zekollari et al. (2019)](https://www.the-cryosphere.net/13/1125/2019/)', width=500)
app = pn.Column(header, title, instruction, fplot, source)
# app.servable(title='Future glacier evolution in the Alps')

In [ ]:
# transform app to model and save it as html-document:
app_to_save = pn.panel(app)
app_to_save.save('alps_future-app.html', embed=True, title='Future glacier evolution in the European Alps')